In [1]:
import pandas as pd
import numpy as np

from IPython.display import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

c:\program files (x86)\python36-32\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('train.csv', encoding='utf8')
df_test = pd.read_csv('test.csv', encoding='utf8')

In [3]:
def preproc_data(df_input):
   
        
    df_output = df_input.copy()
    
    df_output = df_output.drop('_id', axis=1)
    
    df_output['nr.employed'] = df_output['nr.employed'].apply(lambda x: 1 if x <= 5087.65 else 0)
    
    df_output['default'] = df_output['default'].apply(lambda x: 0 if x == 'no' else 1)
    
    df_output['duration_class'] = df_output['duration'].apply(lambda x: 1 if (x >= 197 and x <= 700) else 0)
    
    #df_output['duration_class'] = df_output['duration'].apply(lambda x: 1 if ((df_output['nr.employed'] == 0) & (x <= 518.5)).all() else 0)
    
    #df_output['pdays'] = df_output['pdays'].apply(lambda d: 0 if d == 999 else d)
    
    df_output['age_class'] = df_output['age'].apply(lambda x: 1 if (x >= 25 and x <  50) else 0)
    
    df_output = df_output.drop(['month', 'day_of_week', 'contact', 'pdays'], axis = 1)

    # category code
    df_output = pd.get_dummies(df_output, columns=['job', 'marital', 'education', 'housing', 'loan', 'poutcome'])
        
    return df_output

In [4]:
x = preproc_data(df_train).drop(['target'], axis=1)
y = df_train['target']

x_result = preproc_data(df_test)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [5]:
dtc = DecisionTreeClassifier(max_depth=10)
dtc.fit(x_train, y_train)
roc_auc_score(y_test, dtc.predict_proba(x_test)[:,1])

0.8482870173077641

In [6]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train, y_train)
roc_auc_score(y_test, knn.predict_proba(x_test)[:,1])

0.8652649994392733

In [7]:
#lr = LogisticRegression()
lr = LogisticRegression()#(n_jobs=-1, class_weight='balanced', solver='newton-cg', multi_class='multinomial')
lr.fit(x_train, y_train)
roc_auc_score(y_test, lr.predict_proba(x_test)[:,1])

0.9284443946020708

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)
roc_auc_score(y_test, rf.predict_proba(x_test)[:,1])

0.9374429367126462

In [9]:
x_tmp = x.copy()
x_tmp['dtc'] = dtc.predict_proba(x)[:,1]


x_tmp['rf'] = rf.predict_proba(x)[:,1]
x = x_tmp

x_result_tmp = x_result.copy()
x_result_tmp['dtc'] = dtc.predict_proba(x_result)[:,1]


x_result_tmp['rf'] = rf.predict_proba(x_result)[:,1]
x_result = x_result_tmp

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
lr.fit(x_train, y_train)
roc_auc_score(y_test, lr.predict_proba(x_test)[:,1])

0.9362585323913124

In [11]:
result_predict = pd.DataFrame(data={'_id': df_test['_id'], 'target': lr.predict_proba(x_result)[:,1]})
result_predict_test = pd.DataFrame(data={'_id': df_test['_id'], 'target': lr.predict(x_result)})

In [12]:
result_predict.to_csv('submit_diff.csv', index=False)

In [13]:
result_predict_test[result_predict_test['target'] == 1]['target'].agg('sum')

1489

In [15]:
# from ipywidgets import Image
# from io import StringIO
# import pydotplus
# from sklearn.tree import export_graphviz
# import os 
# os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

# dot_data = StringIO()
# export_graphviz(dtc, feature_names = x.columns, out_file=dot_data, filled=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(value=graph.create_png())